<a href="https://colab.research.google.com/github/ssarker21/Introduction-to-Machine-Learning/blob/main/Homework5_Problem3(b).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# -------------------------
# 0) Reproducibility & device
# -------------------------
torch.manual_seed(0)
device = torch.device("cpu")

# -------------------------
# 1) Load data
# -------------------------
url = "https://github.com/HamedTabkhi/Intro-to-ML/raw/main/Dataset/Housing.csv"
df = pd.read_csv(url)

# Inputs and target (same as before)
cols_X = ["area", "bedrooms", "bathrooms", "stories", "parking"]
col_y  = "price"

X = torch.tensor(df[cols_X].values, dtype=torch.float32)
y = torch.tensor(df[col_y].values,  dtype=torch.float32).unsqueeze(1)  # (N,1)

# -------------------------
# 2) Train/Val split (80/20) — randperm (slide style)
# -------------------------
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)
indices = torch.randperm(n_samples)
train_idx = indices[:-n_val]
val_idx   = indices[-n_val:]

X_train = X[train_idx].to(device)
y_train = y[train_idx].to(device)
X_val   = X[val_idx].to(device)
y_val   = y[val_idx].to(device)

# -------------------------
# 3) Standardize features using TRAIN stats
# -------------------------
x_mean = X_train.mean(dim=0, keepdim=True)
x_std  = X_train.std(dim=0, keepdim=True).clamp_min(1e-8)

X_train_std = (X_train - x_mean) / x_std
X_val_std   = (X_val   - x_mean) / x_std

# -------------------------
# 4) Model: three hidden layers (e.g., 5 -> 32 -> 16 -> 8 -> 1)
# -------------------------
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(5, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )
    def forward(self, x):
        return self.net(x)

model = DeepNet().to(device)

# Utility: count trainable parameters (model size)
def count_params(m):
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

print(f"Model size (trainable parameters): {count_params(model)}")

# -------------------------
# 5) Training setup
# -------------------------
criterion = nn.MSELoss()                 # regression loss
optimizer = optim.Adam(model.parameters(), lr=1e-2)
epochs = 200

def r2_score(y_true, y_pred):
    y_mean = y_true.mean()
    ss_res = torch.sum((y_true - y_pred) ** 2)
    ss_tot = torch.sum((y_true - y_mean) ** 2)
    if ss_tot.item() == 0.0:
        return torch.tensor(0.0, device=y_true.device)
    return 1.0 - ss_res / ss_tot

# -------------------------
# 6) Train (200 epochs)
# -------------------------
start_time = time.time()

for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()

    y_pred_train = model(X_train_std)
    loss_train = criterion(y_pred_train, y_train)
    loss_train.backward()
    optimizer.step()

    if epoch % 50 == 0 or epoch == epochs:
        model.eval()
        with torch.no_grad():
            y_pred_val = model(X_val_std)
            loss_val = criterion(y_pred_val, y_val)
            r2_val = r2_score(y_val, y_pred_val)
        print(f"Epoch {epoch:4d} | Train Loss: {loss_train.item():.4f} "
              f"| Val Loss: {loss_val.item():.4f} | Val R^2: {r2_val.item():.4f}")

train_time_sec = time.time() - start_time

# -------------------------
# 7) Final report
# -------------------------
model.eval()
with torch.no_grad():
    final_train_loss = criterion(model(X_train_std), y_train).item()
    final_val_pred   = model(X_val_std)
    final_val_loss   = criterion(final_val_pred, y_val).item()
    final_val_r2     = r2_score(y_val, final_val_pred).item()

print("\n=== Summary after 200 epochs (3 hidden layers) ===")
print(f"Training time: {train_time_sec:.2f} seconds")
print(f"Final Train Loss (MSE): {final_train_loss:.4f}")
print(f"Final Val  Loss (MSE): {final_val_loss:.4f}")
print(f"Final Val  R^2 (accuracy): {final_val_r2:.4f}")

Model size (trainable parameters): 865
Epoch   50 | Train Loss: 25569970356224.0000 | Val Loss: 28763450507264.0000 | Val R^2: -5.8004
Epoch  100 | Train Loss: 25382247989248.0000 | Val Loss: 28550725894144.0000 | Val R^2: -5.7501
Epoch  150 | Train Loss: 23962050363392.0000 | Val Loss: 26965509668864.0000 | Val R^2: -5.3753
Epoch  200 | Train Loss: 19228027518976.0000 | Val Loss: 21706116694016.0000 | Val R^2: -4.1319

=== Summary after 200 epochs (3 hidden layers) ===
Training time: 0.34 seconds
Final Train Loss (MSE): 19089336565760.0000
Final Val  Loss (MSE): 21706116694016.0000
Final Val  R^2 (accuracy): -4.1319
